# LC-MS vs Fungal Microbiome

In [2]:
import pandas as pd
import numpy as np
import os
import biom

# Visualizations 
import seaborn as sns
import matplotlib.pyplot as plt

import prince

%matplotlib inline

In [3]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/MicrobiomeMetabolome

In [4]:
# set the directory 
os.chdir('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/MicrobiomeMetabolome')

In [5]:
group_colors = {'Climate': '#482677FF', 'Wine Chemistry': '#2D718EFF', 'Plots': '#3CBC75FF', 'Metabolites_Pos':'#FDE725FF', 'Metabolites_Neg':'#DCE318FF'}

# Load and Merge all data

### 1. ITS data 

In [ ]:
%%bash 

# 1. Subset to PostMLF 
qiime feature-table filter-samples \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/mv_filtered_table.qza \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
    --p-where "[sample_type]='Post-MLF'" \
    --o-filtered-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/microvinification/mv_PostMLF_filtered_table.qza

# 2. Rarefy 
qiime feature-table rarefy \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/microvinification/mv_PostMLF_filtered_table.qza \
    --p-sampling-depth 4500 \
    --o-rarefied-table  /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/microvinification/mv_PostMLF_rarefied4500.qza

# 3. Export 
qiime tools export --input-path  /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/microvinification/mv_PostMLF_rarefied4500.qza --output-path mv_PostMLF_rarefied4500

In [ ]:
# export taxonomy
#!qiime tools export --input-path /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza --output-path /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy

In [6]:
# Load the BIOM table
biom_table = biom.load_table("mv_PostMLF_rarefied4500/feature-table.biom")
df_ITS = biom_table.to_dataframe()

# load the metadata to rename the samples accordingly 
ITS_md = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv', sep='\t')
df_ITS.columns = ['PostMLF_' + str(ITS_md.set_index('id').loc[col, 'Year']) + '_Plot' + str(ITS_md.loc[ITS_md['id'] == col, 'Plot'].values[0]) for col in df_ITS.columns]

# load the taxonomy to rename the features
ITS_taxonomy = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy/taxonomy.tsv', sep='\t')
ITS_taxonomy.set_index('Feature ID', inplace=True)
# Rename the index of df_ITS based on the 'Taxon' column of ITS_taxonomy
df_ITS.index = df_ITS.index.map(ITS_taxonomy['Taxon'])

# transpose and reset index
df_ITS = df_ITS.T.reset_index()

df_ITS.head()

,index,k__Fungi;p__Ascomycota,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Hyaloscyphaceae;g__Pseudaegerita;s__Pseudaegerita_sp;sh__SH1255515.10FU,k__Fungi;p__Ascomycota,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__Alternaria_subcucurbitae;sh__SH1206908.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Capnodiales_fam_Incertae_sedis;g__Ramimonilia;s__Ramimonilia_apicalis;sh__SH1394102.10FU,k__Fungi;p__Ascomycota,k__Fungi;p__Fungi_phy_Incertae_sedis;c__Fungi_cls_Incertae_sedis;o__Fungi_ord_Incertae_sedis;f__Fungi_fam_Incertae_sedis;g__Fungi_gen_Incertae_sedis;s__Fungi_sp;sh__SH1213166.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__Alternaria_eureka;sh__SH1206790.10FU,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Sporidiobolales;f__Sporidiobolaceae;g__Rhodotorula;s__Rhodotorula_sp,...,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU,k__Fungi;p__Ascomycota;c__Arthoniomycetes;o__Lichenostigmatales;f__Phaeococcomycetaceae;g__Phaeococcomyces;s__Phaeococcomyces_sp;sh__SH1058115.10FU,k__Fungi;p__Basidiomycota;c__Exobasidiomycetes;o__Microstromatales;f__Microstromataceae;g__Microstroma;s__Microstroma_bacarum,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria,k__Fungi;p__Ascomycota,k__Fungi;p__Ascomycota;c__Lecanoromycetes;o__Teloschistales;f__Teloschistaceae;g__Gallowayella;s__Gallowayella_poeltii,k__Fungi;p__Ascomycota;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycodaceae;g__Hanseniaspora;s__Hanseniaspora_uvarum;sh__SH1339598.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Neophaeothecales;f__Neophaeothecaceae;g__Nothophaeotheca;s__Nothophaeotheca_mirabibensis;sh__SH1394800.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU
0,PostMLF_2023_Plot4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PostMLF_2023_Plot13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PostMLF_2023_Plot9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PostMLF_2023_Plot5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PostMLF_2021_Plot12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Metabolites

In [7]:
df_wMetadata_cleaned_collapsed = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/Pos_MS1MS2_OutlierRemoved.csv', index_col=0)
postMLF_df_Pos = df_wMetadata_cleaned_collapsed[df_wMetadata_cleaned_collapsed['sample_type'] == 'PostMLF'].reset_index()
postMLF_df_Pos.shape

(31, 1946)

In [8]:
df_wMetadata_cleaned_collapsed = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/Neg_MS1MS2_OutlierRemoved.csv', index_col=0)
postMLF_df_Neg = df_wMetadata_cleaned_collapsed[df_wMetadata_cleaned_collapsed['sample_type'] == 'PostMLF'].reset_index()
postMLF_df_Neg.shape

(31, 1393)

In [9]:
postMLF_df_all = pd.merge(postMLF_df_Pos, postMLF_df_Neg, on=('Sample Name', 'sample_type', 'year', 'plot'), how='outer')
postMLF_df_all.shape

(31, 3335)

#### 3. Climate, Plot MD and Wine Chemistry

In [10]:
# CHEMISTRY
md = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/Raw-MD_Vineyard_MV_data.csv')
md["Plot_str"] = "Plot" + md["Plot"].astype(str)
md_wine_chemistry = md[['Plot_str', 'Year', 'Must_Oechsle', 'Wine_Tartatic_Acid', 
                        'Wine_Malic_Acid', 'Wine_Glucose', 'Wine_Fructose', 
                        'Wine_Lactic_Acid', 'Wien_Glycerol', 'Wine_Acetic_Acid']]

df_merged_1 = postMLF_df_all.merge(md_wine_chemistry, left_on=["plot", "year"], right_on=["Plot_str", "Year"], how="inner")
df_merged_1 = df_merged_1.drop(columns=['Plot_str', 'Year', 'sample_type']).dropna()

df_merged_1.head()

,Sample Name,year,plot,(+)-6-epi-stephacidin A,(+)-ochromycinone,(+)-ochromycinone.1,"(-)-11-hydroxy-9,10-dihydrojasmonic acid 11-beta-D-glucoside",(-)-Lamivudine,(-)-Lamivudine.1,(-)-Salsoline,...,α-hydroxyacetovanillone,β-D-Glucopyranuronic acid,Must_Oechsle,Wine_Tartatic_Acid,Wine_Malic_Acid,Wine_Glucose,Wine_Fructose,Wine_Lactic_Acid,Wien_Glycerol,Wine_Acetic_Acid
0,PostMLF_2021_Plot11,2021,Plot11,-0.679276,0.191048,1.709113,-1.337482,-0.806997,-1.008542,-1.265860,...,-0.606316,0.821080,75.0,3.42,0.06,0.33,0.30,1.87,7.13,1.03
1,PostMLF_2021_Plot13,2021,Plot13,-0.620487,0.139359,1.880671,-1.268509,0.079603,-0.809821,-0.281844,...,-0.758207,0.627574,75.0,2.61,0.09,0.33,0.28,1.53,7.13,0.68
2,PostMLF_2021_Plot14,2021,Plot14,-0.258942,0.541058,1.079110,-0.987094,-0.137637,-0.468563,-0.392728,...,-0.601279,0.587509,78.0,3.06,0.06,0.34,0.29,1.47,6.90,0.77
3,PostMLF_2021_Plot15,2021,Plot15,-0.466123,0.648157,1.948666,-1.291637,-0.644351,-0.656881,-0.459233,...,-0.599207,-0.554780,73.0,3.43,0.07,0.33,0.30,1.69,6.29,0.75
4,PostMLF_2021_Plot17,2021,Plot17,-0.594154,0.575244,1.414531,-1.177630,0.921296,1.072237,-1.230299,...,-0.636694,0.808613,73.0,3.06,0.15,0.42,0.29,1.79,7.24,0.81


In [11]:
# PLOTS
md_plots = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/RawMD-Lavaux_plots.csv')
md_plots["Plot_str"] = "Plot" + md_plots["Plot"].astype(str)
md_plots_subset =  md_plots[['Plot_str', 'Altitude', 'Average_slope', 'Exposition', 'Average_radiation', 'Geology', 'Hydromorphie_code']]
# Convert categorical columns to dummy variables
categorical_cols = ['Exposition', 'Geology']
md_plots_subset_dummies = pd.get_dummies(md_plots_subset, columns=categorical_cols, drop_first=True)
# Convert boolean columns to integers (0 or 1)
boolean_columns = md_plots_subset_dummies.select_dtypes(include='bool').columns
md_plots_subset_dummies[boolean_columns] = md_plots_subset_dummies[boolean_columns].astype(int)

df_merged_2 = df_merged_1.merge(md_plots_subset_dummies, left_on=["plot"], right_on=["Plot_str"], how="inner")
df_merged_2 = df_merged_2.drop(columns=['Plot_str']).dropna()

df_merged_2.head()

,Sample Name,year,plot,(+)-6-epi-stephacidin A,(+)-ochromycinone,(+)-ochromycinone.1,"(-)-11-hydroxy-9,10-dihydrojasmonic acid 11-beta-D-glucoside",(-)-Lamivudine,(-)-Lamivudine.1,(-)-Salsoline,...,Wien_Glycerol,Wine_Acetic_Acid,Altitude,Average_slope,Average_radiation,Hydromorphie_code,Exposition_South-West,Geology_Colluvium,Geology_Gravel_moraine,Geology_Molasse
0,PostMLF_2021_Plot11,2021,Plot11,-0.679276,0.191048,1.709113,-1.337482,-0.806997,-1.008542,-1.265860,...,7.13,1.03,490,20,937.5,1,0,0,1,0
1,PostMLF_2021_Plot13,2021,Plot13,-0.620487,0.139359,1.880671,-1.268509,0.079603,-0.809821,-0.281844,...,7.13,0.68,580,30,937.5,1,1,0,0,1
2,PostMLF_2021_Plot14,2021,Plot14,-0.258942,0.541058,1.079110,-0.987094,-0.137637,-0.468563,-0.392728,...,6.90,0.77,440,25,912.5,1,0,0,0,0
3,PostMLF_2021_Plot15,2021,Plot15,-0.466123,0.648157,1.948666,-1.291637,-0.644351,-0.656881,-0.459233,...,6.29,0.75,520,30,912.5,0,1,0,1,0
4,PostMLF_2021_Plot17,2021,Plot17,-0.594154,0.575244,1.414531,-1.177630,0.921296,1.072237,-1.230299,...,7.24,0.81,390,20,912.5,1,1,0,0,1


In [39]:
# CLIMATE
md_climate = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/RawMD-Lavaux-Climate.csv', index_col=0)
# Remove underscores from the 'Plot' column
md_climate['Plot'] = md_climate['Plot'].str.replace('_', '', regex=False)
# include more climate data 
#md_climate_subset = md_climate[['Plot', 'Year', 'cv_rh', 'average_rh', 'cv_temperature', 'accumulated_temperature', 'average_temperature']]
# minimum 
md_climate_subset = md_climate[['Plot', 'Year',  'average_rh','average_temperature']]

df_merged_3 = df_merged_2.merge(md_climate_subset, left_on=["plot", "year"], right_on=["Plot", "Year"], how="inner")
df_merged_3 = df_merged_3.drop(columns=['Plot', 'Year']).dropna()

df_merged_3.head()

,Sample Name,year,plot,(+)-6-epi-stephacidin A,(+)-ochromycinone,(+)-ochromycinone.1,"(-)-11-hydroxy-9,10-dihydrojasmonic acid 11-beta-D-glucoside",(-)-Lamivudine,(-)-Lamivudine.1,(-)-Salsoline,...,Altitude,Average_slope,Average_radiation,Hydromorphie_code,Exposition_South-West,Geology_Colluvium,Geology_Gravel_moraine,Geology_Molasse,average_rh,average_temperature
0,PostMLF_2021_Plot11,2021,Plot11,-0.679276,0.191048,1.709113,-1.337482,-0.806997,-1.008542,-1.265860,...,490,20,937.5,1,0,0,1,0,71.948958,17.964399
1,PostMLF_2021_Plot13,2021,Plot13,-0.620487,0.139359,1.880671,-1.268509,0.079603,-0.809821,-0.281844,...,580,30,937.5,1,1,0,0,1,71.334895,17.544612
2,PostMLF_2021_Plot14,2021,Plot14,-0.258942,0.541058,1.079110,-0.987094,-0.137637,-0.468563,-0.392728,...,440,25,912.5,1,0,0,0,0,70.767447,18.343627
3,PostMLF_2021_Plot15,2021,Plot15,-0.466123,0.648157,1.948666,-1.291637,-0.644351,-0.656881,-0.459233,...,520,30,912.5,0,1,0,1,0,72.104023,17.892713
4,PostMLF_2021_Plot17,2021,Plot17,-0.594154,0.575244,1.414531,-1.177630,0.921296,1.072237,-1.230299,...,390,20,912.5,1,1,0,0,1,72.076768,18.476535


In [40]:
## add microbiome 
df_merged_all = df_merged_3.merge(df_ITS, left_on=["Sample Name"], right_on=["index"], how="inner")
df_merged_all = df_merged_all.drop(columns=['index']).dropna()

df_merged_all.head() 

,Sample Name,year,plot,(+)-6-epi-stephacidin A,(+)-ochromycinone,(+)-ochromycinone.1,"(-)-11-hydroxy-9,10-dihydrojasmonic acid 11-beta-D-glucoside",(-)-Lamivudine,(-)-Lamivudine.1,(-)-Salsoline,...,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU,k__Fungi;p__Ascomycota;c__Arthoniomycetes;o__Lichenostigmatales;f__Phaeococcomycetaceae;g__Phaeococcomyces;s__Phaeococcomyces_sp;sh__SH1058115.10FU,k__Fungi;p__Basidiomycota;c__Exobasidiomycetes;o__Microstromatales;f__Microstromataceae;g__Microstroma;s__Microstroma_bacarum,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria,k__Fungi;p__Ascomycota,k__Fungi;p__Ascomycota;c__Lecanoromycetes;o__Teloschistales;f__Teloschistaceae;g__Gallowayella;s__Gallowayella_poeltii,k__Fungi;p__Ascomycota;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycodaceae;g__Hanseniaspora;s__Hanseniaspora_uvarum;sh__SH1339598.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Neophaeothecales;f__Neophaeothecaceae;g__Nothophaeotheca;s__Nothophaeotheca_mirabibensis;sh__SH1394800.10FU,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Cladosporiales;f__Cladosporiaceae;g__Cladosporium;s__Cladosporium_herbarum;sh__SH1335104.10FU
0,PostMLF_2021_Plot11,2021,Plot11,-0.679276,0.191048,1.709113,-1.337482,-0.806997,-1.008542,-1.265860,...,32.0,0,0,21.0,0,0,0,35.0,7.0,20.0
1,PostMLF_2021_Plot13,2021,Plot13,-0.620487,0.139359,1.880671,-1.268509,0.079603,-0.809821,-0.281844,...,25.0,6.0,0,42.0,0,0,0,52.0,0,19.0
2,PostMLF_2021_Plot14,2021,Plot14,-0.258942,0.541058,1.079110,-0.987094,-0.137637,-0.468563,-0.392728,...,21.0,0,7.0,92.0,0,0,0,26.0,0,0
3,PostMLF_2021_Plot15,2021,Plot15,-0.466123,0.648157,1.948666,-1.291637,-0.644351,-0.656881,-0.459233,...,0,0,0,0,0,0,0,0,0,0
4,PostMLF_2021_Plot17,2021,Plot17,-0.594154,0.575244,1.414531,-1.177630,0.921296,1.072237,-1.230299,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Keep track of the columns' origins
ITS_columns = df_ITS.iloc[:, 1:].columns.tolist()
metabolites_Neg_columns = postMLF_df_Neg.iloc[:, 4:].columns.tolist()
metabolites_Pos_columns = postMLF_df_Pos.iloc[:, 4:].columns.tolist()
wine_chemistry_columns = md_wine_chemistry.iloc[:, 2:].columns.tolist()
plots_columns = md_plots_subset_dummies.iloc[:, 1:].columns.tolist()
climate_columns = md_climate_subset.iloc[:, 2:].columns.tolist()
metadata_columns = ['Sample Name', 'year', 'plot']

In [42]:
# Create a multi-index column structure
multi_index_columns = (
    [('Metabolites_Pos', col) for col in metabolites_Pos_columns] +
    [('Metabolites_Neg', col) for col in metabolites_Neg_columns] +
    [('Wine Chemistry', col) for col in wine_chemistry_columns] +
    [('Plots', col) for col in plots_columns] +
    [('Climate', col) for col in climate_columns] +
    [('Fungi', col) for col in ITS_columns])

# Subset data and assign multi-index
df_metadata = df_merged_all[metadata_columns]  # Keep metadata separately
df_mfa = df_merged_all.drop(columns=metadata_columns)  # Drop metadata
df_mfa.columns = pd.MultiIndex.from_tuples(multi_index_columns)  # Assign multi-index
df_mfa = df_mfa.astype("float64")

# Extract the group names (first level of multi-index)
groups = df_mfa.columns.levels[0].tolist()
groups

['Climate',
 'Fungi',
 'Metabolites_Neg',
 'Metabolites_Pos',
 'Plots',
 'Wine Chemistry']

In [43]:
# Check if there are any NaN values in df_mfa
nan_cells = df_mfa.isna().sum().sum()
print(f"Total NaN cells in df_mfa: {nan_cells}")

Total NaN cells in df_mfa: 0


In [44]:
df_mfa.columns.levels[0]

Index(['Climate', 'Fungi', 'Metabolites_Neg', 'Metabolites_Pos', 'Plots',
       'Wine Chemistry'],
      dtype='object')

# MFA 

## How many Components?

In [45]:
# Define MFA and fit the model
mfa = prince.MFA(
    n_components=n_components,  
    n_iter=3, 
    copy=True,
    check_input=True,
    random_state=42)

In [47]:
mfa = mfa.fit(
    df_mfa,
    groups=groups)

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/prince/mfa.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  all_num = all(pd.api.types.is_numeric_dtype(X[c]) for c in cols)
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/prince/mfa.py:50: PerformanceWarning: indexing past lexsort depth may impact performance.
  all_cat = all(pd.api.types.is_string_dtype(X[c]) for c in cols)


ValueError: Not all columns in "Climate" group are of the same type

In [22]:
# Define the number of components to test
n_components = 10

# Define MFA and fit the model
mfa = prince.MFA(
    n_components=n_components,  
    n_iter=3, 
    copy=True,
    check_input=True,
    random_state=42)

# Fit MFA to the data (excluding metadata columns)
mfa = mfa.fit(
    df_mfa,
    groups=groups,
    supplementary_groups=None)

# Get explained variance per component
explained_variance_ratio = mfa.eigenvalues_ / np.sum(mfa.eigenvalues_)

# Create subplots for explained variance and cumulative variance plots
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Bar plot for variance explained per component
axes[0].bar(range(1, n_components + 1), explained_variance_ratio[:n_components], color='skyblue')
axes[0].set_title('Variance Explained per Component')
axes[0].set_xlabel('Component')
axes[0].set_ylabel('Explained Variance')
axes[0].set_xticks(range(1, n_components + 1))
axes[0].grid(True)

# Line plot for cumulative explained variance (elbow plot)
axes[1].plot(range(1, n_components + 1), np.cumsum(explained_variance_ratio[:n_components]), marker='o', linestyle='-')
axes[1].set_title('Cumulative Explained Variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance')
axes[1].set_xticks(range(1, n_components + 1))
axes[1].grid(True)

# Adjust layout to ensure proper display
plt.tight_layout()
plt.show()

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/prince/mfa.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  all_num = all(pd.api.types.is_numeric_dtype(X[c]) for c in cols)
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/prince/mfa.py:50: PerformanceWarning: indexing past lexsort depth may impact performance.
  all_cat = all(pd.api.types.is_string_dtype(X[c]) for c in cols)


ValueError: Not all columns in "Climate" group are of the same type